<a href="https://colab.research.google.com/github/jabo16ac/Empirical-analysis-of-investors-perceptions-of-CoCos/blob/main/Data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Empirical analysis of investors’ perceptions of CoCos: Are they going concern capital?

## 1 Imports
Libraries are imported

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## 2 Processing
Data is prepared for statistical analysis

### 2.1 Financial data
The financial data is imported and processed

In [53]:
# Import financial data data
df = pd.read_csv('03_Financials.csv', header=0)

# Set unique identifier (ISIN+Year) as index
df = df.set_index('UID')

# Exclude banks with assets less than EUR 1bn 
df = df[df['TOTAL ASSETS'] >= 10**9]

# Rename columns
df.columns = ['ISIN', 'Year', 'DERIVATIVES (ASSETS)', 'OTHER SECURITIES',
       'TOTAL ASSETS', 'DERIVATIVES (LIABILITIES AND EQUITY)',
       'OFF BALANCE SHEET ITEMS', '10_TotalCapitalRatio', 'TOTAL CAPITAL',
       'TIER 1 CAPITAL', '9_CostIncomeRatio',
       '2_NPLRatio', '8_ROA',
       '3_LoansRatio', 'Country']

# Re-format and calculate columns
df['RWA'] = df['TOTAL CAPITAL']/(df['10_TotalCapitalRatio']/100)
df['1_RWARatio'] = df['RWA']/df['TOTAL ASSETS']
df['Derivs'] = (df['DERIVATIVES (ASSETS)'] + df['DERIVATIVES (LIABILITIES AND EQUITY)'])
df['4_SecuritiesRatio'] = df['OTHER SECURITIES']/df['TOTAL ASSETS']
df['5_DerivativesRatio'] = df['Derivs']/df['TOTAL ASSETS']
df['2_NPLRatio'] = df['2_NPLRatio']/100
df['3_LoansRatio'] = df['3_LoansRatio']/100
df['6_OBSRatio'] = df['OFF BALANCE SHEET ITEMS']/df['TOTAL ASSETS']
df['7_Size'] = np.log(df['TOTAL ASSETS'])
df['8_ROA'] = df['8_ROA']/100
df['9_CostIncomeRatio'] = df['9_CostIncomeRatio']/100
df['10_TotalCapitalRatio'] = df['10_TotalCapitalRatio']/100
df['11_CapitalQuality'] = df['TIER 1 CAPITAL']/df['TOTAL CAPITAL']

# Drop old columns
df = df.drop(['DERIVATIVES (ASSETS)', 'OTHER SECURITIES', 'TOTAL ASSETS', 
         'DERIVATIVES (LIABILITIES AND EQUITY)', 'OFF BALANCE SHEET ITEMS', 
         'TOTAL CAPITAL', 'TIER 1 CAPITAL', 'RWA', 'Derivs',], axis = 1)

# Select these columns
columns = ['ISIN', 'Year', 'Country', '1_RWARatio', '2_NPLRatio', '3_LoansRatio', 
           '4_SecuritiesRatio', '5_DerivativesRatio', '6_OBSRatio', '7_Size', 
           '8_ROA', '9_CostIncomeRatio', '10_TotalCapitalRatio', '11_CapitalQuality']
financials = df[columns]

### 2.2 Risk data
The stock price data is imported and volatility, VaR, and ES is calculated.

In [55]:
### Import price and stock data
pdf = pd.read_csv('01_Stock price.csv', header=3) 
ddf = pd.read_csv('02_Dates.csv', header=3)

### Extract bank ISINs
banks = pdf.columns.values # Deduct bank names

### Calculate daily returns
rdf = (pdf - pdf.shift(1)) / pdf.shift(1) # Calculate returns

### Define variables
voldf = pd.DataFrame()
years = pd.Series(np.arange(2011, 2021))

### Run loop for every bank
for j, element in enumerate(banks):
  name = element 
  names = pd.Series(name, index = np.arange(0,10)) # List names x years
  
  # Clear variables 
  vols = [] 
  VaRs = []
  ESs = []

  ### Run loop for every every (by every bank)
  for i in range(2011, 2021):
    # Rename columns and join price and date
    y = rdf[banks[j]].rename('Return') 
    x = ddf[banks[j]].rename('Date')
    df = pd.concat([x,y], axis = 1).set_index(pd.to_datetime(x, dayfirst=True)).drop('Date', axis = 1)

    # Select relevant year
    year = i
    start = str(str(year) + "-01-01")
    end = str(str(year) + "-12-31")
    df = df[start:end]

    # Calculate daily volatility and mean
    daily_vol = df.std(ddof = 1)
    mean = np.mean(df)

    # Calculate annualsed volatility, VaR and ES
    vol = np.sqrt(len(df)) * daily_vol # Annualize given trading days where len(df) is the no. of trading days
    VaR = mean + daily_vol*scipy.stats.norm.ppf(0.025)
    ES = mean - daily_vol*(scipy.stats.norm.pdf(scipy.stats.norm.ppf(0.025)))/0.025
    
    # Append the yearly vol/VaR/ES to table for each bank
    vols = pd.Series(np.append(vols, [vol]))
    VaRs = pd.Series(np.append(VaRs, [VaR]))
    ESs  = pd.Series(np.append(ESs, [ES]))

  # Create dataframe with unique identifier
  d1 = pd.concat([names, years], axis = 1)

  # Set index to unique idenfier by joining columns
  d2 = d1[[0,1]].astype(str).apply(lambda x: '-'.join(x), axis=1) # Create index (bank + year)
  
  # Join all three columns (UID, ISIN, Year)
  df = pd.concat([d2, d1], axis = 1) 
  
  # Join above with calculated volatitilies, VaRs, and ES
  volatilities = pd.concat([df, vols, VaRs, ESs], axis = 1) 
  
  # Append to all previous volatility tables for each bank
  voldf = voldf.append(volatilities) 

# Rename columns and set UID as index
voldf.columns =  ['UID', 'ISIN', 'Year', '12_Volatility', '13_VaR', '14_ES']
voldf = voldf.set_index('UID')

### 2.3 Financial data
The financial data is imported and processed

In [63]:
### Load CoCo issuance data
ccc = pd.read_csv('04_CoCos.csv', header=0).set_index('UID')

### Merge financials, risk-metrics, and CoCo data
df = pd.merge(financials, voldf.iloc[:,2:], left_index=True, right_index=True)
df = pd.merge(df, ccc, left_index=True, right_index=True)

### 2.4 Data manipulation
Control variables are lagged

In [57]:
### Pre-define dataframe
shiftedcontrols = pd.DataFrame()

### Get unique bank names
banks = df['ISIN'].unique()

### Declare columns to shift
shiftcols = ['7_Size',
            '3_LoansRatio',
            '8_ROA',
            '9_CostIncomeRatio', 
            '10_TotalCapitalRatio', 
            '11_CapitalQuality']

### Lag control variables
for j, element in enumerate(banks):
  bankdf = df[df['ISIN']==element]
  bankdf[shiftcols] = bankdf[shiftcols].shift(1)
  shiftedcontrols = shiftedcontrols.append(bankdf).sort_index()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


### 2.5 Outlier exclusion
Control variables are lagged

In [ ]:
### Declare columns to drop NA's in, then drop them
dropcols   = ['1_RWARatio',
              '3_LoansRatio',
              '7_Size',
              '8_ROA',
              '9_CostIncomeRatio', 
              '10_TotalCapitalRatio', 
              '11_CapitalQuality',
              '12_Volatility',
              '13_VaR',
              '14_ES']
df = shiftedcontrols.dropna(subset=dropcols)

### Manual outlier exclusion
df = df[df['1_RWARatio'] < 1]
df = df[df['2_NPLRatio'] < 1]
df = df[df['3_LoansRatio'] < 1]
df = df[df['4_SecuritiesRatio'] < 1]
df = df[df['5_DerivativesRatio'] < 1]
df = df[df['6_OBSRatio'] < 1]
df = df[df['8_ROA'] < 0.5]
df = df[df['9_CostIncomeRatio'] < 1.5]
df = df[df['10_TotalCapitalRatio'] < .5]
df = df[df['11_CapitalQuality'] < 1.5]
df = df[df['12_Volatility'] < 2]

### Exclude all banks which do not have atleast five years of data
selection = df['ISIN'].value_counts()>4
df = df[df['ISIN'].isin(selection.index[selection].values)]

### 2.6 Descriptive statistics

Net sample is tabulated across country and year

In [69]:
bank_year_obs = pd.pivot_table(df, values='ISIN', index=['Country'], columns=['Year'], aggfunc=len, fill_value=0)
#print(bank_year_obs.to_latex(index=True)) # Used to export to Latex
bank_year_obs

Year,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0
Country,,,,,,,,,
CH,1,1,0,0,0,0,0,0,0
DE,1,1,1,1,1,1,1,1,1
ES,1,1,1,1,1,1,1,1,1
FR,3,3,3,3,3,3,3,3,3
GB,4,4,4,4,4,3,3,3,3
IT,1,1,1,1,0,0,1,0,0
NL,1,1,1,1,0,0,1,1,1


Averages and total for above table are calculated

In [83]:
print("Total obs. per country:\n", bank_year_obs.sum(axis=1))
print("Average obs. per country:", bank_year_obs.sum(axis=1).sum())

print("\nTotal obs. per year:\n", bank_year_obs.sum(axis=0))
print("Average obs. per year:", bank_year_obs.sum(axis=1).mean())

print("\nTotal obs:", bank_year_obs.sum().sum())

Total obs. per country:
 Country
CH     2
DE     9
ES     9
FR    27
GB    32
IT     5
NL     7
dtype: int64
Average obs. per country: 91

Total obs. per year:
 Year
2011.0    12
2012.0    12
2013.0    11
2014.0    11
2015.0     9
2016.0     8
2017.0    10
2018.0     9
2019.0     9
dtype: int64
Average obs. per year: 13.0

Total obs: 91


CoCo sample is tabulated across country and year

In [81]:
### Dataframe is pre-defined
CoCos = pd.DataFrame()

### Year and country is selected from main data frame
CoCos["Country"] = df.index.str.slice(0,2)
CoCos["Year"] = df.index.str.slice(13,13+4).astype('int32')

### CoCo indicator variables are extracted from main data frame
CoCos["CoCo_EC"] = df['CoCo_EC'].values
CoCos["CoCo_PW"] = df['CoCo_PW'].values
CoCos["CoCo"] = df['CoCo'].values

### EC CoCos
#CoCo_obs = pd.pivot_table(CoCos, values='CoCo_EC', index=['Country'], columns=['Year'], aggfunc=sum, fill_value=0)
#print(CoCo_obs.to_latex(index=True))

### WD CoCos
#CoCo_obs = pd.pivot_table(CoCos, values='CoCo_PW', index=['Country'], columns=['Year'], aggfunc=sum, fill_value=0)
#print(CoCo_obs.to_latex(index=True))

### All CoCos
CoCo_obs = pd.pivot_table(CoCos, values='CoCo', index=['Country'], columns=['Year'], aggfunc=sum, fill_value=0)
#print(CoCo_obs.to_latex(index=True)) # Used to export to Latex
CoCo_obs

Year,2011,2012,2013,2014,2015,2016,2017,2018,2019
Country,,,,,,,,,
CH,0,0,0,0,0,0,0,0,0
DE,0,0,0,1,0,0,0,0,0
ES,0,0,0,0,0,0,0,0,0
FR,0,0,2,2,2,3,1,2,3
GB,0,0,0,1,1,0,0,1,1
IT,0,0,0,1,0,0,1,0,0
NL,0,0,0,0,0,0,0,0,1


Descriptive statistics for all continous variables are calculated

In [67]:
#print(df.describe().transpose().to_latex(float_format="%.4f")) # Used to export to Latex
df.describe()

,Year,1_RWARatio,2_NPLRatio,3_LoansRatio,4_SecuritiesRatio,5_DerivativesRatio,6_OBSRatio,7_Size,8_ROA,9_CostIncomeRatio,10_TotalCapitalRatio,11_CapitalQuality,12_Volatility,13_VaR,14_ES,Pre-CoCo,CoCo,Post-CoCo,Pre-CoCo_EC,CoCo_EC,Post-CoCo_EC,Pre-CoCo_PW,CoCo_PW,Post-CoCoPW
count,91.000000,86.000000,89.000000,91.000000,91.000000,91.000000,89.000000,91.000000,91.000000,91.000000,86.000000,86.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000
mean,2014.714286,0.302521,0.047234,0.392441,0.206216,0.295737,0.196574,28.163508,0.001986,0.708922,0.160741,0.841542,0.330713,-0.040565,-0.048401,0.318681,0.252747,0.186813,0.329670,0.263736,0.219780,0.296703,0.252747,0.208791
std,2.625879,0.085453,0.029408,0.139946,0.059019,0.207130,0.066151,0.294816,0.003675,0.131038,0.025449,0.070210,0.127447,0.015818,0.018821,0.468547,0.436995,0.391921,0.472698,0.443099,0.416392,0.459335,0.436995,0.408697
min,2011.000000,0.150791,0.007100,0.193700,0.089024,0.053351,0.035463,27.631021,-0.016100,0.509400,0.106000,0.661448,0.158010,-0.086906,-0.103201,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2012.000000,0.246210,0.025400,0.284600,0.157357,0.120132,0.160248,28.006021,0.000600,0.626900,0.140250,0.799841,0.236891,-0.048382,-0.057798,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2014.000000,0.288193,0.041100,0.342100,0.203008,0.240741,0.188889,28.137839,0.002700,0.671400,0.160500,0.843317,0.289719,-0.035485,-0.042404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2017.000000,0.359431,0.062900,0.551450,0.259070,0.440931,0.225926,28.426271,0.004100,0.778650,0.182500,0.884700,0.402623,-0.029473,-0.035173,1.000000,0.500000,0.000000,1.000000,1.000000,0.000000,1.000000,0.500000,0.000000
max,2019.000000,0.499298,0.155500,0.688600,0.302532,0.785714,0.396040,28.660641,0.007000,1.125200,0.223000,1.000000,0.691394,-0.019623,-0.023509,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 3 Analysis

### 3.1 Asset risk-shifting test

#### 3.1.1 All CoCos
Regressions are run for the six asset risk-shifiting metrics against all CoCos

In [61]:
res1 = smf.ols(formula='Q("1_RWARatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res1.summary())

res2 = smf.ols(formula='Q("2_NPLRatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res2.summary())

res3 = smf.ols(formula='Q("3_LoansRatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res3.summary())

res4 = smf.ols(formula='Q("4_SecuritiesRatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res4.summary())

res5 = smf.ols(formula='Q("5_DerivativesRatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res5.summary())

res6 = smf.ols(formula='Q("6_OBSRatio") ~ Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res6.summary())

                            OLS Regression Results                            
Dep. Variable:        Q("1_RWARatio")   R-squared:                       0.901
Model:                            OLS   Adj. R-squared:                  0.849
Method:                 Least Squares   F-statistic:                     17.25
Date:                Fri, 12 Mar 2021   Prob (F-statistic):           2.45e-13
Time:                        00:26:08   Log-Likelihood:                 138.82
No. Observations:                  59   AIC:                            -235.6
Df Residuals:                      38   BIC:                            -192.0
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

#### 3.1.2 EC CoCos
Regressions are run for the six asset risk-shifiting metrics against EC CoCos

In [84]:
res1 = smf.ols(formula='Q("1_RWARatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res1.summary())

res2 = smf.ols(formula='Q("2_NPLRatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res2.summary())

res3 = smf.ols(formula='Q("3_LoansRatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res3.summary())

res4 = smf.ols(formula='Q("4_SecuritiesRatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res4.summary())

res5 = smf.ols(formula='Q("5_DerivativesRatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res5.summary())

res6 = smf.ols(formula='Q("6_OBSRatio") ~ Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res6.summary())

                            OLS Regression Results                            
Dep. Variable:        Q("1_RWARatio")   R-squared:                       0.943
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     39.49
Date:                Fri, 12 Mar 2021   Prob (F-statistic):           1.71e-28
Time:                        00:53:25   Log-Likelihood:                 212.98
No. Observations:                  86   AIC:                            -374.0
Df Residuals:                      60   BIC:                            -310.2
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

#### 3.1.3 PW CoCos

Regressions are run for the six asset risk-shifiting metrics against PW CoCos

In [ ]:
res1 = smf.ols(formula='Q("1_RWARatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res1.summary())

res2 = smf.ols(formula='Q("2_NPLRatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res2.summary())

res3 = smf.ols(formula='Q("3_LoansRatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res3.summary())

res4 = smf.ols(formula='Q("4_SecuritiesRatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res4.summary())

res5 = smf.ols(formula='Q("5_DerivativesRatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res5.summary())

res6 = smf.ols(formula='Q("6_OBSRatio") ~ Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res6.summary())

### 3.2 Primary empirical model

#### 3.2.1. All CoCos
Regressions are run for the three risk-metrics against all CoCos

In [85]:
res1 = smf.ols(formula='Q("12_Volatility") ~ Q("Pre-CoCo") + Q("CoCo") + Q("Post-CoCo") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res1.summary())

res2 = smf.ols(formula='Q("12_Volatility") ~ Q("Pre-CoCo_PW") + Q("CoCo_PW") + Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res2.summary())

res3 = smf.ols(formula='Q("12_Volatility") ~ Q("Pre-CoCo_EC") + Q("CoCo_EC") + Q("Post-CoCo_EC") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + ISIN + C(Year)', 
               data=df).fit()
print(res3.summary())


#res4 = smf.ols(formula='Q("12_Volatility") ~ Q("Pre-CoCo_EC") + Q("Post-CoCo_EC") + Q("Pre-CoCo_PW") + Q("CoCo_PW") + Q("Post-CoCoPW") + Q("7_Size") + Q("8_ROA") + Q("9_CostIncomeRatio") + Q("10_TotalCapitalRatio") + Q("11_CapitalQuality") + Q("3_LoansRatio") + ISIN + C(Year)', 
#               data=df).fit()
#print(res4.summary())



                            OLS Regression Results                            
Dep. Variable:     Q("12_Volatility")   R-squared:                       0.901
Model:                            OLS   Adj. R-squared:                  0.855
Method:                 Least Squares   F-statistic:                     19.60
Date:                Fri, 12 Mar 2021   Prob (F-statistic):           2.84e-20
Time:                        00:54:03   Log-Likelihood:                 155.68
No. Observations:                  86   AIC:                            -255.4
Df Residuals:                      58   BIC:                            -186.6
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

#### 3.2.2. WD CoCos

Regressions are run for the three risk-metrics against WD CoCos

#### 3.2.3. EC CoCos

Regressions are run for the three risk-metrics against EC CoCos